In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",100, "display.width",2000, "display.max_colwidth",100)

from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import re

#### Read Data

In [3]:
input_df = pd.read_csv("protac.csv").reset_index(drop=True)
input_df = input_df.rename(columns={"E3 ligase":"E3ligase"})
print("input_df: {:,} x {:,}".format(*input_df.shape), "(input df) \n")

print("Counts of distinct values in columns")
for x in ["Target","E3ligase"]:
  tmp = input_df.groupby(x, as_index=False).size().sort_values("size", ascending=False, ignore_index=True)
  print(f"col:{x}  (#unique:{tmp.shape[0]:,})")
  print(tmp.head(20).T, "\n")

del tmp

input_df: 3,939 x 84 (input df) 

Counts of distinct values in columns
col:Target  (#unique:289)
         0     1     2    3     4    5    6        7     8     9       10    11    12     13     14    15          16                17   18    19
Target   ER  BRD4  CDK4   AR  CDK6  BTK  ALK  BCR-ABL  BRD3  BRD2  BCL-xL  MEK1  CDK2  PARP1  HDAC6  BRD9  BRAF V600E  EGFR L858R/T790M  FAK  CDK9
size    182   164   143  141   139  106   97       96    77    77      77    76    70     63     60    53          53                52   52    52 

col:E3ligase  (#unique:12)
            0     1      2     3     4    5    6       7       8       9       10    11
E3ligase  CRBN   VHL  cIAP1  XIAP  MDM2  AhR  IAP  DCAF15  DCAF16  RNF114  DCAF11  RNF4
size      2571  1152    122    34    30    9    5       4       4       4       3     1 



E3Ligase Classes

In [4]:
for i in range(len(input_df["E3ligase"])):
    if 'IAP' in str(input_df.loc[i].at["E3ligase"]):
        input_df.at[i,'E3ligase']='IAP'
    elif 'DCAF' in str(input_df.loc[i].at["E3ligase"]):
        input_df.at[i,'E3ligase']='DCAF'
    elif 'RNF' in str(input_df.loc[i].at["E3ligase"]):
        input_df.at[i,'E3ligase']='RNF'

tmp = input_df.groupby('E3ligase', as_index=False).size().sort_values("size", ascending=False, ignore_index=True)
print(f"col:{'E3ligase'}  (#unique:{tmp.shape[0]:,})")
print(tmp.head(20).T, "\n")
del tmp


col:E3ligase  (#unique:7)
             0     1    2     3     4    5    6
E3ligase  CRBN   VHL  IAP  MDM2  DCAF  AhR  RNF
size      2571  1152  161    30    11    9    5 



In [5]:
print('Number of Rows with explicit DC50: ' + str((input_df.shape[0] - input_df['DC50 (nM)'].isna().sum())))
#input_df = input_df.dropna(subset=['DC50 (nM)',"Percent degradation (%)"],how='all')
input_df = input_df.dropna(subset=['DC50 (nM)'])
print("input_df: {:,} x {:,}".format(*input_df.shape), "(input df) \n")

Number of Rows with explicit DC50: 638
input_df: 638 x 84 (input df) 



#### Features

Target and E3Ligase

In [6]:
model_df = pd.concat([
    pd.get_dummies(input_df["Target"], prefix="tgt"),
    pd.get_dummies(input_df["E3ligase"], prefix="e3")
  ], axis=1).reset_index(drop=True)

print("model_df: {:,} x {:,}".format(*model_df.shape), "(target and E3ligase OHE features)")

model_df: 638 x 89 (target and E3ligase OHE features)


Molecular Fingerprint

In [7]:
bits = 256

def get_morgan_fp(smiles:str, nbits:int=bits):
  m = Chem.MolFromSmiles(smiles)
  fingerprint = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=nbits)
  array = np.zeros((0,), dtype=np.int8)
  DataStructs.ConvertToNumpyArray(fingerprint, array)
  return array

smiles = [get_morgan_fp(x).tolist() for x in input_df["Smiles"]]
smiles_df = pd.DataFrame(smiles, columns=["sm"+str(n) for n in range(1,bits+1)])
print("model_df: {:,} x {:,}".format(*model_df.shape))

#assert model_df.columns == smiles_df.columns

model_df = pd.concat([model_df,smiles_df],axis=1)
print("smiles_df: {:,} x {:,}".format(*smiles_df.shape))
print("model_df: {:,} x {:,}".format(*model_df.shape), "(added smiles features)")

del smiles, smiles_df

model_df: 638 x 89
smiles_df: 638 x 256
model_df: 638 x 345 (added smiles features)


Cell Type

In [8]:
celltypes = []
for i in range(input_df.shape[0]):
    row = str(input_df.iloc[i].to_list())
    cell = re.findall('[\w\.;/-]+ cells', row)
    if len(cell)==0:
        celltypes.append('Unknown')
    else:
        celltypes.append(cell[0][:-6])

print('Cell Types: ' + str(set(celltypes)))
len(set(celltypes))
model_df['ct'] = celltypes
model_df = pd.get_dummies(model_df, prefix='ct')
model_df
print("\nmodel_df: {:,} x {:,}".format(*model_df.shape), "(added cell type feature)")

Cell Types: {'MCF-7/T47D', 'HLE/HuH-7/SNU-423', 'Jurkat/MOLT-4/U937', '11', 'Sk-Mel-28', 'Ramos/THP-1', 'NCI-H661/NCI-H838', 'PC3', 'NAMALWA/XLA', 'MOLT-4/platelets', 'HLF/SNU-398/HUCCT1', 'Kasumi-1/NB4/MV-4-11', 'Namalwa/CA-46', 'SW620', 'Sertoli/germ', 'OCI-AML2/OCI-AML3/Ramos', '22Rv1/VCaP/LnCaP95', 'Molm-16/SU-DHL-1', 'A549', 'Mino', 'HeLa/DB', 'NAMALWA', 'HeLa/HEK293', 'MM.1S/MDA-MB-231', 'HCC827', 'MDA-MB-436/Capan-1/SW620', 'HeLa', 'THP1', 'TM3/PA1/MDA-MB-436', 'MDA-MB-231', 'KYSE-270', 'MCF7', 'SR', '4935', 'MV-4-11/NCI-H1568', 'SRD15', 'MDA-MB-468', 'Z138/HBL-1/Mino', 'HT-29/SK-MEL-28', 'KYSE520/MV4;11', '264.7', 'Ramos', 'HCC-827', '22RV1', '205', 'Huh7.5', 'WI38', 'A431', 'MCF7/MDA-MB-231', 'HCT116', 'VCaP', 'MM', 'Hs578t', '22Rv1', 'EOL-1/A-204', 'SK-MEL-28', 'MM.1S/MV-4-11/MDA-MB-468', 'Unknown', 'H3122/Karpas299/Kelly', 'HUH-1/HepG2/SK-Hep-1', 'NCI-H23/NCI-H358', 'U251', 'LNCaP', 'LNCaP/Ramos', 'H1975', 'XLA', 'PBMC', 'MOLT-4', 'EOL-1', 'MOLM-14', 'RI-1', 'KM12/HEL', 'MM1

109

,tgt_ABL1,tgt_ADRA1A,tgt_AKT1,tgt_AKT2,tgt_AKT3,tgt_ALK,tgt_ALK G1202R,tgt_AR,tgt_AR-V7,tgt_AURKA,tgt_BCL-xL,tgt_BCL2,tgt_BCR-ABL,tgt_BLK,tgt_BRAF G466V,tgt_BRAF G469A,tgt_BRAF V600E,tgt_BRD2,tgt_BRD3,tgt_BRD4,tgt_BRD4 BD1,tgt_BRD7,tgt_BRD9,tgt_BTK,tgt_BTK C481S,tgt_CD147,tgt_CDK19,tgt_CDK2,tgt_CDK4,tgt_CDK6,tgt_CDK8,tgt_CDK9,tgt_CRBN,tgt_Cdc20,tgt_EED,tgt_EGFR,tgt_EGFR L858R,tgt_EGFR L858R/T790M,tgt_EGFR e19d,tgt_ER,tgt_ERRalpha,tgt_EZH2,tgt_FAK,tgt_FKBP12,tgt_GCN5,tgt_GSK-3beta,tgt_GSK3A,tgt_HDAC3,tgt_HDAC6,tgt_HMGCR,...,ct_MOLT-4/platelets,ct_MV-4-11,ct_MV-4-11/NCI-H1568,ct_MV4-11,ct_MV4;11,ct_Mino,ct_Molm-16,ct_Molm-16/SU-DHL-1,ct_NAMALWA,ct_NAMALWA/XLA,ct_NCI-H2030,ct_NCI-H2030/MIAPaCa-2/SW1573,ct_NCI-H23/NCI-H358,ct_NCI-H661/NCI-H838,ct_Namalwa/CA-46,ct_OCI-AML2/OCI-AML3/Ramos,ct_OVCAR8/HeLa,ct_PBMC,ct_PC-3,ct_PC-3/LNCaP/MDA-MB-231,ct_PC3,ct_PC3-S1,ct_PC9,ct_Pfeiffer/DOHH2/K562,ct_RI-1,ct_Ramos,ct_Ramos/THP-1,ct_SK-MEL-246,ct_SK-MEL-28,ct_SR,ct_SRD15,ct_SU-DHL-1/NCI-H2228,ct_SUM149,ct_SW1573,ct_SW480,ct_SW620,ct_Sertoli/germ,ct_Sk-Mel-28,ct_T47D/MB-MDA-231,ct_THP-1,ct_THP1,ct_TM3/PA1/MDA-MB-436,ct_Tu-I,ct_U251,ct_Unknown,ct_VCaP,ct_WI38,ct_XLA,ct_Z138/HBL-1/Mino,ct_hPBMC
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
634,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
635,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
636,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0



model_df: 638 x 454 (added cell type feature)


#### Response Variable (Binary)

In [9]:
# dfstr = input_df.select_dtypes(include=[object])
# rows = dfstr.apply(lambda x: x.str.contains("degradation",case=False)).any(axis=1)
# model_df.insert(0, "resp", rows.values.astype(int))

# print("model_df: {:,} x {:,}".format(*model_df.shape), "(added response variable)")
# del dfstr, rows

#### Response Variable (DC50)

In [10]:
#add and process DC50 column

model_df['resp'] = list(input_df['DC50 (nM)'])
model_df['resp'] = model_df["resp"].str.replace("[<=>]","")
model_df['resp'] = model_df["resp"].str.replace("\(n/a\)","0")
model_df['resp'] = model_df["resp"].str.replace('[\d]+[-~]',"")
model_df = model_df.loc[~model_df['resp'].str.contains('[a-zA-Z]')]

/var/folders/vs/8ts0hp5d0wzf8zdrzyqly4840000gn/T/ipykernel_82926/1778872113.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  model_df['resp'] = model_df["resp"].str.replace("[<=>]","")
/var/folders/vs/8ts0hp5d0wzf8zdrzyqly4840000gn/T/ipykernel_82926/1778872113.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  model_df['resp'] = model_df["resp"].str.replace("\(n/a\)","0")
/var/folders/vs/8ts0hp5d0wzf8zdrzyqly4840000gn/T/ipykernel_82926/1778872113.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  model_df['resp'] = model_df["resp"].str.replace('[\d]+[-~]',"")


Split DC50 by Cell Type

In [19]:
out_nochange = []
out_v1 = []
out_v2 = []
for row in model_df.to_dict(orient="records"):
  #no slashes
  if "/" not in row["resp"]:
    out_nochange.append(row)
    continue
  
  #slashes in resp matched by a column with ct which has same number of slashes
  lst = [k for k,v in row.items() if k.startswith("ct") and "/" in k and v==1]
  assert len(lst) in [0,1]
  if len(lst) == 1:
    #test.append((lst[0],row["resp"]))
    cols = lst[0].split("/")
    vals = row["resp"].split("/")
    assert len(cols)==len(vals)
    assert len(cols) in [2,3]
    
    #adding 2 or 3 rows
    for c,v in zip(cols,vals):
      dct = row.copy()
      k = c if c.startswith("ct_") else "ct_"+c
      dct[k] = 1
      dct["resp"] = v
      del dct[lst[0]]
      out_v1.append(dct)
  else:
    vals = row["resp"].split("/")
    dct = row.copy()
    dct["resp"] = vals[0]
    out_v2.append(dct)
    
    
model_df = pd.DataFrame(out_nochange+out_v1+out_v2)
model_df = model_df.fillna(0)
model_df

,tgt_ABL1,tgt_ADRA1A,tgt_AKT1,tgt_AKT2,tgt_AKT3,tgt_ALK,tgt_ALK G1202R,tgt_AR,tgt_AR-V7,tgt_AURKA,tgt_BCL-xL,tgt_BCL2,tgt_BCR-ABL,tgt_BLK,tgt_BRAF G466V,tgt_BRAF G469A,tgt_BRAF V600E,tgt_BRD2,tgt_BRD3,tgt_BRD4,tgt_BRD4 BD1,tgt_BRD7,tgt_BRD9,tgt_BTK,tgt_BTK C481S,tgt_CD147,tgt_CDK19,tgt_CDK2,tgt_CDK4,tgt_CDK6,tgt_CDK8,tgt_CDK9,tgt_CRBN,tgt_Cdc20,tgt_EED,tgt_EGFR,tgt_EGFR L858R,tgt_EGFR L858R/T790M,tgt_EGFR e19d,tgt_ER,tgt_ERRalpha,tgt_EZH2,tgt_FAK,tgt_FKBP12,tgt_GCN5,tgt_GSK-3beta,tgt_GSK3A,tgt_HDAC3,tgt_HDAC6,tgt_HMGCR,...,ct_NCI-H661,ct_NCI-H838,ct_OVCAR8,ct_LnCaP95,ct_Namalwa,ct_CA-46,ct_SU-DHL-1,ct_H3122,ct_Karpas299,ct_Kelly,ct_platelets,ct_NCI-H1568,ct_DB,ct_KYSE520,ct_Sertoli,ct_germ,ct_TM3,ct_PA1,ct_MDA-MB-436,ct_MB-MDA-231,ct_Hep3B2.1-7,ct_SNU-387,ct_HUH-1,ct_HepG2,ct_SK-Hep-1,ct_HLF,ct_SNU-398,ct_HUCCT1,ct_HLE,ct_HuH-7,ct_SNU-423,ct_MIAPaCa-2,ct_NCI-H23,ct_NCI-H358,ct_Capan-1,ct_HCC1937,ct_KM12,ct_HEL,ct_HT-29,ct_U937,ct_Kasumi-1,ct_NB4,ct_OCI-AML2,ct_OCI-AML3,ct_Z138,ct_HBL-1,ct_Pfeiffer,ct_DOHH2,ct_H1666,ct_CAL-12T
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
707,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
708,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
709,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

DC50 Distribution and Cutoff Determination

In [39]:
#dropping rows with outlier DC50 values
model_df.drop(model_df.index[model_df['resp'].astype(float) <= 0], inplace=True)
model_df.drop(model_df.index[model_df['resp'].astype(float) > 100], inplace=True)
print("model_df: {:,} x {:,}".format(*model_df.shape), "(DC50 values filtered)")
print("25th percentile: " + str(model_df['resp'].astype(float).quantile(.25)))
print("50 percentile: " + str(model_df['resp'].astype(float).quantile(.50)))
print("75 percentile: " + str(model_df['resp'].astype(float).quantile(.75)))
for  index, row in model_df.iterrows():
    if index!=0:
        if float(row['resp'])<=4.0:
            row['resp'] = 0
        elif float(row['resp'])<=15.6:
            row['resp'] = 1
        elif float(row['resp'])<50.0:
            row['resp']=2
        else:
            row['resp']=3

model_df: 421 x 506 (DC50 values filtered)
25th percentile: 4.0
50 percentile: 15.6
75 percentile: 50.0


#### Model Matrices

In [40]:
train,test = train_test_split(model_df, random_state=1, test_size=.2)

features = train.columns.tolist()[1:]  #dropping "resp"
print(f"nfeatures: {len(features):,}")

X_train = train.drop("resp", axis=1).values
y_train_true = train["resp"].values

X_test = test.drop("resp", axis=1).values
y_test_true = test["resp"].values

del train, test

nfeatures: 505


In [41]:
def eval_classifier(clf):
  y_train_pred = clf.predict(X_train)
  y_test_pred = clf.predict(X_test)

  stats = [
    ["train",roc_auc_score(y_train,y_train_pred), average_precision_score(y_train,y_train_pred)],
    ["test",roc_auc_score(y_test,y_test_pred), average_precision_score(y_test,y_test_pred)]
  ]
  
  with pd.option_context("display.float_format", "{:,.2%}".format):
    print(pd.DataFrame(stats, columns=["","AUC","AP"]))

#### Model: RandomForest
```
min_samples_leaf  minimum number of samples required to be at a leaf node (incrementing to reduce overfitting)
```

In [42]:
# for msl in [10,5,2]:
#   print(f"min_samples_leaf:{msl}")
#   clf = RandomForestClassifier(random_state=10, n_estimators=50, min_samples_leaf=msl).fit(X_train,y_train)
#   eval_classifier(clf)
  
# del clf, msl

clf = RandomForestClassifier(random_state=10, n_estimators=50, min_samples_leaf=10).fit(X_train,y_train_true)
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)
r2_train = r2_score(y_train_true, y_train_pred)
r2_test = r2_score(y_test_true, y_test_pred)
print(r2_train, r2_test)

-0.715968351738256 -1.4303847622241244


In [43]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
mse = mean_squared_error(y_test_true, y_test_pred)
mse

2030.1065417529412

In [44]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test_true,y_test_pred))
print(classification_report(y_test_true,y_test_pred))
print(accuracy_score(y_test_true, y_test_pred))

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

        0.08       0.00      0.00      0.00         1
       0.228       0.00      0.00      0.00         1
       0.263       0.00      0.00      0.00         1
         0.3       0.00      0.00      0.00         1
        0.37       0.00      0.00      0.00         1
       0.384       0.00      0.00      0.00         0
         0.4       0.29      1.00      0.44         2
        0.57       0.00      0.00      0.00         1
         0.6       0.00      0.00      0.00         1
         0.7       0.00      0.00      0.00         1
         0.8       0.00      0.00      0.00         1
        0.86       0.00      0.00      0.00         1
           1       0.00      0.00      0.00         0
         1.1       0.00      0.00      0.00         1
        1.26       0.00      0.00      0.00         1
         1.5   

/Users/divyanori/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/divyanori/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/divyanori/miniconda3/envs/my-rdkit-env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modi